In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

from hdfs.ext.kerberos import KerberosClient

import mysql.connector
from mysql.connector import Error
from sqlalchemy import create_engine


import pandas as pd
import os

# # Setup WebDriver
driverOptions = webdriver.ChromeOptions()
driverOptions.browser_version = "120"
driverOptions.add_argument("--headless") # Runs Chrome in headless mode.
driverOptions.add_argument('--no-sandbox') # # Bypass OS security model, MUST BE THE VERY FIRST OPTION
driverOptions.add_argument('--disable-dev-shm-usage') # overcome limited resource problems
driverOptions.add_argument('start-maximized') # 
driverOptions.add_argument('disable-infobars')
driverOptions.add_argument('--disable-extensions')
driverOptions.add_argument("--remote-debugging-port=9222")
driverOptions.add_argument("--verbose")
driverOptions.add_argument("--log-level=0")

# # Setup WebDriver
driverOptions = webdriver.ChromeOptions()
driverOptions.browser_version = "120"
driver = webdriver.Chrome(options=driverOptions)

#Scrape the table data
table_data = []
# Create DataFrame for the scraped data
df = pd.DataFrame()  
#Create headers
headers = []
#add all the pages urls
pages = []

#initializing hdfs client
hdfs_client = KerberosClient('http://master-node:9870','OPTIONAL')


#Page for American Market Indexes
pageIndexesAmerica = 'https://www.google.com/finance/markets/indexes/americas'
#Page for EMEA Market Indexes
pageIndexesEMEA = 'https://www.google.com/finance/markets/indexes/europe-middle-east-africa'
#Page for Asia Market Indexes
pageIndexesAsia = 'https://www.google.com/finance/markets/indexes/asia-pacific'
#Page for most active stocks and funds
pageMostActive = 'https://www.google.com/finance/markets/most-active'
#page for Gainers
pageGainers = 'https://www.google.com/finance/markets/gainers'
#Page for Loosers
pageLoosers = 'https://www.google.com/finance/markets/losers'
#Page for Climate Leaders
pageClimateLeaders = 'https://www.google.com/finance/markets/climate-leaders'
#Page for Crypto currencies
pageCrypto = 'https://www.google.com/finance/markets/cryptocurrencies'
#Page for Currencies
pageCurrencies = 'https://www.google.com/finance/markets/currencies'
##<span jsname="V67aGc" class="VfPpkd-vQzf8d" aria-hidden="true">Accept all</span>

#function to get EMEA Indices
def getStockMarketData(filename,url):
    # List to store data
    data = []
    driver.get(url)
    driver.delete_all_cookies()
    AcceptRejectButton()
    # print("Indices page opened :", IndicesRegion)
    try:
        # Locate the UL/LI elements (adjust the selector as per your needs)
        list_items = WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "ul > li")))
        # print(list_items)  
        # Iterate over each LI element
        for item in list_items:
                # Adjust the selector based on your specific HTML structure
                Name = item.find_element(By.CLASS_NAME, "ZvmM7").text
                Value = item.find_element(By.CLASS_NAME, "Bu4oXd").text
                ChangeValue =  item.find_element(By.CLASS_NAME, "P2Luy").text
                ChangePercentage = item.find_element(By.CLASS_NAME, "JwB6zf").text
                # print("IndexName:", IndexName)
                # print("IndexValue:", IndexValue)
                # print("IndexChangeValue:", IndexChangeValue)
                # print("IndexChangePercentage:", IndexChangePercentage)
                # Append data to the list as a dictionary
                data.append({   
                    'name': str.strip(Name),
                    'value': str.strip(Value),
                    'changevalue': str.strip(ChangeValue),
                    'changepercentage': str.strip(ChangePercentage)
                })
    except:
        print("Element not found")        
    # Create a DataFrame
    df = pd.DataFrame(data)
    df['value'] = df['value'].str.replace(',', '')
    df['changevalue'] = df['changevalue'].str.replace(',', '')
    df['changepercentage'] = df['changepercentage'].str.replace('%','') 
    # Display the DataFrame
    # print(df)
    df.dropna(how='all', inplace=True)
    create_csv(df,filename)
    writeToHDFS(df,filename)
    readFromCSV(filename)
    # print(df)
    # return df
    
def closeDriver():
    driver.close() 
    driver.quit()
    
      
def AcceptRejectButton():
    try:
        # Find the button using XPath
        buttonReject_xpath = "//button[contains(@class, 'VfPpkd-LgbsSe') and @aria-label='Reject all']"
        buttonReject = driver.find_element(By.XPATH, buttonReject_xpath)
        buttonReject.click()
        print()    
    except:
        print()

#function to create csv files from scraped data
def create_csv(data,fileName):
    # Create a DataFrame
    df = pd.DataFrame(data)
    # Display the DataFrame
    # print(df)
    df.dropna(how='all', inplace=True)
    # # #Save DataFrame to CSV
    csvFile = fileName + '.csv'
    df.to_csv(csvFile, index=False)
    # return df


# Call fucntion to get data from all three pages of Stock-Market Indexes of region : Europe, Asia and America
df_asian= getStockMarketData('asian-indices',pageIndexesAsia)
df_american= getStockMarketData('american-indices',pageIndexesAmerica)
df_emea= getStockMarketData('emea-indices',pageIndexesEMEA)
df_gainer= getStockMarketData('gainer-stocks',pageGainers)
df_loosers= getStockMarketData('looser-stocks',pageLoosers)
df_crypto= getStockMarketData('crypto-currencies',pageCrypto)
df_climate_leaders= getStockMarketData('climateleaders',pageClimateLeaders)
df_currencies= getStockMarketData('currencies',pageCurrencies)

# def extraDataCleanUp():
#     ##Clean the data. Section starts -------------------
#     ####REmove € symobol from data frame of Indexes
#     df_gainer['value'] = df_gainer['value'].str.replace(r'[€]', '', regex=True)
#     df_loosers['changevalue'] = df_loosers['changevalue'].str.replace(r'[€]', '', regex=True)


###Clean section ends ------------------------------

##Write the data to hdfs
def writeToHDFS(dataFrame,filename):
    df = pd.DataFrame(dataFrame)
    filePath = '/data/'+ filename +'.csv'
    with hdfs_client.write(filePath, encoding='utf-8') as writer:
        df.to_csv(writer)

def readFromCSV(filename):
    # Read from HDFS
    filepath = '/data/' + filename+'.csv'
    with hdfs_client.read(filepath) as reader:
        df2 = pd.read_csv(reader, sep=',')
    print(df2)

#close the driver
closeDriver()

In [ ]:
# Create a bar graph from the columns 'IndexName' and 'IndexValue'
# Use the data from 'IndexChangeValue' to set the color of the bars
import plotly.express as px
import plotly.io as pio
import pandas as pd
# Read the CSV file into a DataFrame again
df = pd.read_csv('Currencies.csv')
print(df)
# Define colors based on 'ChangeValue'
# df['color'] = df['IndexChangeValue'].apply(lambda x: 'green' if x > 0 else 'red')

# Creating the bar plot
# Create the bar graph
fig = px.bar(df, x='Name', y='Value', color='color',
            #  color_discrete_map={'red': 'red', 'green': 'green'},
             labels={'Currency Exchange Difference': 'Currency Pair'},
             title='Bar Graph of Index Name vs Index Value')

# Show the graph
fig.show()